# ESA with Verbal Fluency 

## ESA FINN

In [6]:
#Define misc variables
ID = "0001"
textfile = "/Users/au183362/Documents/postdoc/Parkinson-DBS/Eira_Aksnes/raw/test.txt"
iterations = 30000 # input to Semantic-module

#From Yund et al.:
#ESA switch threshold is "PercentRESA * MeanAllESA" or the (Minimum sequence ESA)+0.00001 
#so that there is at least one ESA switch.
PercentRESA = 0.75


In [2]:
from dasem.semantic import Semantic

### Example from DASEM, semantic relatedness

#Example:

  >>> semantic = Semantic(30000)  # and wait
    >>> semantic.relatedness(['hund', 'kat', 'mus', 'fisk']).round(3)
    array([[ 1.   ,  0.022,  0.005,  0.001],
           [ 0.022,  1.   ,  0.002,  0.   ],
           [ 0.005,  0.002,  1.   ,  0.01 ],
           [ 0.001,  0.   ,  0.01 ,  1.   ]])

In [3]:
!python -m dasem.wikipedia download --verbose

2019-09-22 22:04:23,957 - __main__ - INFO - Downloading https://dumps.wikimedia.org/dawiki/latest/dawiki-latest-pages-articles.xml.bz2 to /Users/au183362/dasem_data/wikipedia/dawiki-latest-pages-articles.xml.bz2


In [4]:
import time
t = time.time()

#On Eira's macbook:
#Both Semantic(300) and Semantic(30) = 600-700 sec ≈ 10-12 min
#On AH's macbook: Semantic(30000) = 500 sec ≈ 8 min
semantic = Semantic(iterations)

elapsed = time.time()-t
print(elapsed)


500.948143005


In [5]:
#cd /Users/eiraaksnes/Documents/CFIN/Raw_Data

/Users/eiraaksnes/Documents/CFIN/Raw_Data


In [7]:
inputs = [line.rstrip() for line in open(textfile)]
print(inputs)

['Hest', 'Hund', 'Kat', 'Mus']


In [ ]:
#Clean inputs for duplicates (and unknowns? - can't do that until we've run semantic.relatedness...)
#need to figure out what semantic.relatedness does to "unknown" concepts/words

In [8]:
semantic.relatedness(inputs).round(3)

array([[1.   , 0.007, 0.003, 0.002],
       [0.007, 1.   , 0.02 , 0.006],
       [0.003, 0.02 , 1.   , 0.01 ],
       [0.002, 0.006, 0.01 , 1.   ]])

In [31]:
ESA_table = semantic.relatedness(inputs).round(3)
for n, i in enumerate(inputs):     
    if n > 0:
        print("{0:s}-{1:s} = {2:2.3f}".format(inputs[n-1], i, ESA_table[n, n-1]))
#print(ESA[1,0])

Hest-Hund = 0.007
Hund-Kat = 0.020
Kat-Mus = 0.010


In [35]:
#Extract ESA values for continuous (i.e., chronological) pairs
ESA = []
DictESA = {}
SumESA = 0.0

for i in range(len(inputs)-1):
    OneESA = ESA_table[i,i+1]
    DictESA["{0:s}-{1:s}".format(inputs[i], inputs[i+1])] = OneESA 
    ESA = ESA + [OneESA]
    #ESA.append(OneESA)
    SumESA = SumESA + OneESA
    
MeanESA = SumESA / len(ESA)

print(DictESA)
print("\nMean (chronological pairwise) ESA for Participant ID XYZ: {0:2.3f}".format(MeanESA.round(3)))


{'Kat-Mus': 0.01, 'Hund-Kat': 0.02, 'Hest-Hund': 0.007}

Mean (chronological pairwise) ESA for Participant ID XYZ: 0.012


In [38]:
#Extract ESA values for all possible pairs
DictAllESA = {}
N_AllESA = 0
SumAllESA = 0.0

for i in range(len(inputs)-1):
    for j in range(i+1, len(inputs)):
            OneESA = ESA_table[i,j]
            DictAllESA["{0:s}-{1:s}".format(inputs[i], inputs[j])] = OneESA
            N_AllESA = N_AllESA + 1
            SumAllESA = SumAllESA + OneESA
        
if (N_AllESA == 0):
    MeanAllESA = 0.0
    SOI = 0.0
else:
    MeanAllESA = SumAllESA / N_AllESA
    if MeanAllESA == 0.0:
        SOI = 0.0
    else:
        SOI = MeanESA / MeanAllESA

print(inputs)
print(ESA_table)
print(DictAllESA)
print("\nMean All ESA for Participant ID XYZ: {0:2.3f}".format(MeanAllESA.round(3)))
print("\nSOI Participant ID XYZ: {0:2.3f}".format(SOI.round(3)))


['Hest', 'Hund', 'Kat', 'Mus']
[[1.    0.007 0.003 0.002]
 [0.007 1.    0.02  0.006]
 [0.003 0.02  1.    0.01 ]
 [0.002 0.006 0.01  1.   ]]
{'Hest-Mus': 0.002, 'Hest-Hund': 0.007, 'Hund-Kat': 0.02, 'Kat-Mus': 0.01, 'Hest-Kat': 0.003, 'Hund-Mus': 0.006}

Mean All ESA for Participant ID XYZ: 0.008

SOI Participant ID XYZ: 1.542


## Clusters and switches

In [56]:
# Setting the ESA_Threshold
ESA.sort()
ESA_Threshold = PercentRESA * MeanAllESA  # ESA switch if ESA < ESA_Threshold
if ESA_Threshold < ESAsort[0]:
    ESA_Threshold = ESAsort[0] + 0.000001
print(ESA_Threshold)

0.007001


NameError: name 'SumCs' is not defined

In [61]:
ESAsw = [int(esa<ESA_Threshold) for esa in ESA]
print(ESAsw)

[1, 0, 0]


In [ ]:
#ESA switches
ESAsw = [int(esa<ESA_Threshold) for esa in ESA]


In [68]:
##### Yund-code
#ESAswitch 
ESAsw = []  # Initialize ESA switch array.
ESAcs = []  # Initialize ESA cluster array.
for i in range(len(inputs)-1):
    ESAcs = ESAcs + [' ']
    if ESA[i] < ESA_Threshold:
        ESAsw = ESAsw + [1]
    else:
        ESAsw = ESAsw + [0]
        
# There is one ESAcs for each word, but one less ESAsw
ESAcs = ESAcs + [' ']
iiESAsw = len(ESAsw) - 1
ii = 1
#  SumCS and NumCS to compute mean multi-word cluster size.
SumCS = 0.0
NumCS = 0.0
for i in range(iiESAsw, -1, -1):
    if (ESAsw[i] == 0):
        ii = ii + 1
    else:
        #		print (i, ii)
        ESAcs[i+1] = str(ii)
        if (ii > 1):
            SumCS = SumCS + ii
            NumCS = NumCS + 1
        ii = 1
ESAcs[0] = str(ii)
if (ii > 1):
    SumCS = SumCS + ii
    NumCS = NumCS + 1
#
if NumCS == 0:
    NumCS = 1
sumESAsw = sum(ESAsw)
if sumESAsw == 0:
    sumESAsw = 1
    
#print(NumCs)

NameError: name 'NumCs' is not defined

## Create/save output file

In [ ]:
#Create output file for each subject (.txt)
#NB! not all entries in the below code are relevant for our current analysis

#create/open file for writing output
outfile = open('output_' + ID +'_Out.txt', 'w')

#summary values
outfile.write('Number of animals:  ')
outfile.write(format(N_Totl, '>3d'))
outfile.write('\nNumber correct:      ')
outfile.write(format(N_Uniq, '>3d'))
outfile.write('\nRepetitions:        ')
outfile.write(format(N_Dups, '>3d'))
outfile.write('\nUnknown Animals:    ')
outfile.write(format(N_UNKN, '>3d'))
outfile.write('\n\nMean Log Freq:      ')
outfile.write(format(statistics.mean(LogFreq), '10.4f'))
outfile.write('  (')
outfile.write(format(statistics.stdev(LogFreq), '6.4f'))
outfile.write(')')
outfile.write('\nMean Syllables:     ')
outfile.write(format(statistics.mean(Syllabs), '10.4f'))
outfile.write('  (')
outfile.write(format(statistics.stdev(Syllabs), '6.4f'))
outfile.write(')')
outfile.write('\nMean Typicality     ')
outfile.write(format(statistics.mean(Typical), '10.4f'))
outfile.write('  (')
outfile.write(format(statistics.stdev(Typical), '6.4f'))
outfile.write(')')

#ESA analysis
outfile.write('\n\nSequence Mean ESA:  ')
outfile.write(format(MeanESA, '10.4f'))
outfile.write('\nTotal Mean ESA:     ')
outfile.write(format(MeanAllESA, '10.4f'))
outfile.write('\nSem Org Index (SOI):')
outfile.write(format(SOI, '10.4f'))
outfile.write('\n\nESA switches:       ')
outfile.write(format(sum(ESAsw), '>4d'))
outfile.write('  (')
outfile.write(format(N_Totl/sumESAsw, '5.2f'))
outfile.write(' words/switch)')
outfile.write('  (')
outfile.write(format(SumCS/NumCS, '5.2f'))
outfile.write(' words/cluster)')

## ESA William

In [ ]:
ESA = []
    #
    for i in range(len(AnimalIndex)):
        if i == 0:
            ESA = []
            SumESA = 0.0
        elif ('UNKNOWN' in VF_Animals[AnimalIndex[i]]) | ('UNKNOWN' in VF_Animals[AnimalIndex[i-1]]):
            ESA = ESA + [0.0000000]
        else:
            OneESA = ESA_2s[AnimalIndex[i]][AnimalIndex[i-1]]
            ESA = ESA + [OneESA]
            SumESA = SumESA + OneESA

    #
    MeanESA = SumESA / len(ESA)
    #
    N_AllESA = 0
    SumAllESA = 0.0
    #
    tmp = 0
    for i in range(0, (len(AnimalIndex)-1)):
        for j in range(i+1, len(AnimalIndex)):
            if (Dups[i] == 1) | (Dups[j] == 1):
                tmp = tmp + 1
            elif ('UNKNOWN' in VF_Animals[AnimalIndex[i]]) | ('UNKNOWN' in VF_Animals[AnimalIndex[j]]):
                N_AllESA = N_AllESA + 1
            else:
                OneESA = ESA_2s[AnimalIndex[i]][AnimalIndex[j]]
                N_AllESA = N_AllESA + 1
                SumAllESA = SumAllESA + OneESA

    #
    if (N_AllESA == 0):
        MeanAllESA = 0.0
        SOI = 0.0
    else:
        MeanAllESA = SumAllESA / N_AllESA
        if MeanAllESA == 0.0:
            SOI = 0.0
        else:
            SOI = MeanESA / MeanAllESA
    #
    outfile.write('\n\nSequence Mean ESA:  ')
    outfile.write(format(MeanESA, '10.4f'))
    outfile.write('\nTotal Mean ESA:     ')
    outfile.write(format(MeanAllESA, '10.4f'))
    outfile.write('\nSem Org Index (SOI):')
    outfile.write(format(SOI, '10.4f'))
    #
    ESAsort = []		# Need independent ESA list to sort, for quartiles.
    for i in range(len(ESA)):
        ESAsort = ESAsort + [ESA[i]]

    #
    ESAsort.sort()
    ESA_Threshold = PercentRESA * MeanAllESA  # ESA switch if ESA < ESA_Threshold
    if ESA_Threshold < ESAsort[0]:
        ESA_Threshold = ESAsort[0] + 0.000001
    #
    ESAsw = []  # Initialize ESA switch array.
    ESAcs = []  # Initialize ESA cluster array.
    for i in range(len(AnimalIndex)-1):
        ESAcs = ESAcs + [' ']
        if ESA[i] < ESA_Threshold:
            ESAsw = ESAsw + [1]
        else:
            ESAsw = ESAsw + [0]

    # There is one ESAcs for each word, but one less ESAsw
    ESAcs = ESAcs + [' ']
    iiESAsw = len(ESAsw) - 1
    ii = 1
    #  SumCS and NumCS to compute mean multi-word cluster size.
    SumCS = 0.0
    NumCS = 0.0
    for i in range(iiESAsw, -1, -1):
        if (ESAsw[i] == 0):
            ii = ii + 1
        else:
            #		print (i, ii)
            ESAcs[i+1] = str(ii)
            if (ii > 1):
                SumCS = SumCS + ii
                NumCS = NumCS + 1
            ii = 1
    ESAcs[0] = str(ii)
    if (ii > 1):
        SumCS = SumCS + ii
        NumCS = NumCS + 1
    #
    if NumCS == 0:
        NumCS = 1
    sumESAsw = sum(ESAsw)
    if sumESAsw == 0:
        sumESAsw = 1
    outfile.write('\n\nESA switches:       ')
    outfile.write(format(sum(ESAsw), '>4d'))
    outfile.write('  (')
    outfile.write(format(N_Totl/sumESAsw, '5.2f'))
    outfile.write(' words/switch)')
    outfile.write('  (')
    outfile.write(format(SumCS/NumCS, '5.2f'))
    outfile.write(' words/cluster)')